In [14]:
from nltk import word_tokenize
import nltk
import re
import collections
import numpy as np
import pickle
import glob, os
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree
from nltk import word_tokenize
import nltk
import re
import collections
import numpy as np
import pickle
import numpy
import os
import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Embedding, Bidirectional, GRU, Concatenate, Permute, Dot, Multiply
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix,f1_score
from sklearn.metrics import classification_report,precision_recall_fscore_support
from keras.models import model_from_json
from sklearn.model_selection import train_test_split


from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

from gensim.models.wrappers import FastText

from sklearn.metrics import accuracy_score


from nmt_utils import *


# fix random seed for reproducibility
numpy.random.seed(7)

## Part 1 Load data

In [15]:
working_dir = '/Users/ennoh/Desktop/Detector/data/'

raw_training = working_dir+'ALLREPORTS/'
labels_training = working_dir+'ShAReCLEFeHealthAA_1stRoundRS_CLEFPIPEDELIMITED_200_CUIs/'
raw_testing =  working_dir+'ALLREPORTS2/'
#labels_testing =working_dir+'Task2ReferenceStd_CLEFShARe2013Test_StrictAndLenientpipe/'
labels_testing =working_dir+ 'Task2ReferenceStd_CLEFShARe2013Test_TestSpansOnlypipe/'


output = working_dir+'output_2/'
train_bio = output+"train_bio/"
test_bio = output+"test_bio/"
train_tags_dict = output+'train_tags_dict'
test_tags_dict = output+'test_tags_dict'

train_doc_file = output+'train_docs.pkl'
train_lable_file = output+'train_labels.pkl'
train_lable_index_file = output+'train_label_index.pkl'

test_doc_file = output+'test_docs.pkl'
test_lable_file = output+'test_labels.pkl'
test_lable_index_file = output+'test_label_index.pkl'

label_dict_file = output+'label_dict.pickle'

SLIDING_WINDOW = 4

In [16]:
def context(l, size=7):
    """
    Wraps up the input string.
    """
    l = list(l)
    #左右一起补(size/2)个0
    lpadded = size // 2 * [0] + l + size // 2 * [0]
    
    out = [lpadded[i:(i + size)] for i in range(len(l))]
    return out

In [17]:
#Loads in both train and test files

#Training
fo = open(train_doc_file,'rb')
training_file_instances = pickle.load(fo)

fo = open(train_lable_index_file,'rb')
training_label_index_instances = pickle.load(fo)

fo = open(train_lable_file,'rb')
training_label_instances = pickle.load(fo)

#Testing
fo = open(test_doc_file,'rb')
test_file_instances = pickle.load(fo)

fo = open(test_lable_index_file,'rb')
test_label_index_instances = pickle.load(fo)

fo = open(test_lable_file,'rb')
test_label_instances = pickle.load(fo)

fo = open(label_dict_file,'rb')
label_dict = pickle.load(fo)
inv_label_dict = {v: k for k, v in label_dict.items()}

In [18]:
#Convert training examples into a single file
X_trn=[]
for s in training_file_instances:
    X_trn += context(s,size=SLIDING_WINDOW)
X_trn = np.array(X_trn)

y_trn=[]
for s in training_label_index_instances:
    y_trn += list(s)
y_trn = np.array(y_trn)

#Convert testing examples into a single file
X_tst=[]
for s in test_file_instances:
    X_tst += context(s,size=SLIDING_WINDOW)
X_tst = np.array(X_tst)

y_tst=[]
for s in test_label_index_instances:
    y_tst += list(s)
y_tst = np.array(y_tst)

In [19]:
m = X_trn.shape[0]

In [20]:
y_trn_seq=[]
for s in training_label_index_instances:
    y_trn_seq += context(s,size=SLIDING_WINDOW)
y_trn_seq = np.array(y_trn_seq)

y_tst_seq=[]
for s in test_label_index_instances:
    y_tst_seq += context(s,size=SLIDING_WINDOW)
y_tst_seq = np.array(y_tst_seq)

### Encode X

In [21]:
X_trn

array([['0', '0', '17830', '||||'],
       ['0', '17830', '||||', '23590'],
       ['17830', '||||', '23590', '||||'],
       ...,
       ['end', 'of', 'report', '¤'],
       ['of', 'report', '¤', '¤'],
       ['report', '¤', '¤', '0']], dtype='<U30')

In [22]:
MAX_SEQUENCE_LENGTH = 4
train_texts = [' '.join(text) for text in X_trn]
test_texts = [' '.join(text) for text in X_tst]

all_texts = train_texts + test_texts
t = Tokenizer()
t.fit_on_texts(all_texts)
word_to_index = t.word_index
vocab_len = len(word_to_index) + 1
encoded_X_train = t.texts_to_sequences(train_texts)
encoded_X_test = t.texts_to_sequences(test_texts)

#padding
encoded_X_train = pad_sequences(encoded_X_train, maxlen=MAX_SEQUENCE_LENGTH)
encoded_X_test = pad_sequences(encoded_X_test, maxlen=MAX_SEQUENCE_LENGTH)

In [23]:
encoded_X_train

array([[    0,    20,    20, 10765],
       [    0,    20, 10765,  6142],
       [    0,     0, 10765,  6142],
       ...,
       [  199,     6,    69,     1],
       [    6,    69,     1,     1],
       [   69,     1,     1,    20]], dtype=int32)

In [25]:
print("X_trn.shape:", X_trn.shape)
print("y_trn.shape:", y_trn.shape)
print("y_trn_seq.shape:", y_trn_seq.shape)

#print("Xoh.shape:", Xoh.shape)
#print("Yoh.shape:", Yoh.shape)

X_trn.shape: (115916, 4)
y_trn.shape: (115916,)
y_trn_seq.shape: (115916, 4)


In [26]:
index = 40
print("Source date:", X_trn[index])
print("Source after preprocessing (indices):", encoded_X_train[index])
print()
print("Target date:", y_trn[index])
print("Target date (seq):", y_trn_seq[index])
#print()
#print("Source after preprocessing (indices):", encoded_X_train[index])
#print("Target after preprocessing (indices):", encoded_X_test[index])
#print()
#print("Source after preprocessing (one-hot):", Xoh[index])
#print("Target after preprocessing (one-hot):", Yoh[index])

Source date: ['04' 'sex' 'f' '¤']
Source after preprocessing (indices): [ 96 251 360   1]

Target date: 1
Target date (seq): [0 0 1 0]


In [27]:
y_trn

array([0, 0, 0, ..., 0, 0, 0])

### Encode Y

In [28]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y]
    return Y

In [29]:
a = [0,1,2,3]
convert_to_one_hot(a, 4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [30]:
b = [0,0,1,0]
convert_to_one_hot(b, 2)

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [31]:
Yoh_train = np.array(list(map(lambda x: to_categorical(x, num_classes=2), y_trn_seq)))
Yoh_test = np.array(list(map(lambda x: to_categorical(x, num_classes=2), y_tst_seq)))

In [44]:
Yoh_train.shape

(115916, 4, 2)

In [45]:
Yoh_test.shape

(108856, 4, 2)

In [75]:
index = 40
print("Source data:", X_trn[index])
print("Source after preprocessing (indices):", encoded_X_train[index])
print()
print("Target data:", y_trn[index])
print("Target data (seq):", y_trn_seq[index])
print("Target data (one-hot):", Yoh_train[index])

Source data: ['04' 'sex' 'f' '¤']
Source after preprocessing (indices): [ 96 251 360   1]

Target data: 1
Target data (seq): [0 0 1 0]
Target data (one-hot): [[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [76]:
index = 40
print("Source data:", X_tst[index])
print("Source after preprocessing (indices):", encoded_X_test[index])
print()
print("Target data:", y_tst[index])
print("Target data (seq):", y_tst_seq[index])
print("Target data (one-hot):", Yoh_test[index])

Source data: ['20' 'sex' 'f' '¤']
Source after preprocessing (indices): [ 98 251 360   1]

Target data: 1
Target data (seq): [0 0 1 0]
Target data (one-hot): [[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


## Part 2 Training

### 2.1 Embedding layer

In [77]:
mimic_model = FastText.load_fasttext_format('mimic_3_fasttext.model.bin')

In [78]:
def pretrained_embedding_layer():
    
    #emb_dim = 200
    #len(word_to_index) = 11053
    #vocab_len = 11054
    
    emb_dim = mimic_model["ca"].shape[0] 
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        if word in mimic_model.wv.vocab:
            emb_matrix[index, :] = mimic_model[word]
        
    embedding_layer = Embedding(vocab_len, emb_dim)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
            
    return embedding_layer

In [79]:
y = pretrained_embedding_layer()

In [80]:
y.get_weights()[0][3][8]

0.21576422

### 2.2 Attention layer

In [81]:
# Defined shared layers as global variables
Tx = 4
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [82]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e. (≈1 lines)
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    
    return context

### 2.3 Model

In [83]:
n_a = 64
n_s = 128
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
#output_layer = Dense(len(machine_vocab), activation=softmax)
output_layer = Dense(2, activation=softmax)

In [84]:
def model(Tx, Ty, n_a, n_s):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"      embedding_dim
    machine_vocab_size -- size of the python dictionary "machine_vocab"      label_number

    Returns:
    model -- Keras model instance
    """
    
    
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    
    sentence_indices = Input(shape=(Tx,), dtype=np.int32)
    
    embedding_layer = pretrained_embedding_layer()
    
    embeddings = embedding_layer(sentence_indices) 
    
    #X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
        
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(embeddings)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model([sentence_indices, s0, c0], outputs)
    
    
    return model

In [85]:
model = model(4, 4, n_a, n_s)

In [86]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 4, 200)       2210800     input_1[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 4, 128)       135680      embedding_2[0][0]                
__________________________________________________________________________________________________
repeat_vec

In [87]:
opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [88]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh_train.swapaxes(0,1))

In [89]:
model.fit([encoded_X_train, s0, c0], outputs, epochs=1, batch_size=100)

Epoch 1/1
115916/115916 [==============================] - 90s 774us/step - loss: 0.1499 - dense_2_loss: 0.0352 - dense_2_acc: 0.9868 - dense_2_acc_1: 0.9840 - dense_2_acc_2: 0.9864 - dense_2_acc_3: 0.9883


## Test on random input

In [97]:
EXAMPLES = ['she has the ca', 'she is working hard', '04 sex f ¤', 'has the ca but', 'the pt is not', 'pt is not good']

EXAMPLES = t.texts_to_sequences(EXAMPLES)
EXAMPLES = pad_sequences(EXAMPLES, maxlen=4)

print(EXAMPLES)
PREDICTION = model.predict([EXAMPLES, s0, c0])

#print(PREDICTION)
PREDICTION = np.asarray(PREDICTION)
print(PREDICTION.shape) #(4, 3, 2)
PREDICTION = PREDICTION.swapaxes(0,1)
print(PREDICTION.shape) #(3, 4, 2)

PR = np.argmax(PREDICTION, axis = -1)
print(PR.shape)

#print('User Input: ' , E)
print('Prediction: ')
print(PR)

[[  27  100    3 2013]
 [  27   14 2113 3346]
 [  96  251  360    1]
 [ 100    3 2013  118]
 [   3   84   14   83]
 [  84   14   83  282]]
(4, 6, 2)
(6, 4, 2)
(6, 4)
Prediction: 
[[0 0 0 1]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]


### Test on testset

In [91]:
PREDICTION_ = model.predict([encoded_X_test, s0, c0])

#print(PREDICTION_)
PREDICTION_ = np.asarray(PREDICTION_)
print(PREDICTION_.shape) #(4, 108856, 2)
PREDICTION_ = PREDICTION_.swapaxes(0,1)
print(PREDICTION_.shape) #(108856, 4, 2)

PR_ = np.argmax(PREDICTION_, axis = -1)
print(PR_.shape)

print(PR_)

(4, 108856, 2)
(108856, 4, 2)
(108856, 4)
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 ...
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [92]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [93]:
a = np.asarray([[0,0,0,0],[0,0,0,1]])
b = np.asarray([[0,0,0,0],[0,0,0,0]])

In [94]:
accuracy_score(y_true=a,y_pred=b)

0.5

In [95]:
accuracy_score(PR_,y_tst_seq)

0.9555743367384435

In [229]:
f1_score(PR_,y_tst_seq,average='micro')

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

## Test on document

In [246]:
fo = open('task2_train_dict','rb')
task2_test_dict = pickle.load(fo)

In [247]:
filename = 'data/ALLREPORTS2/00176-102920-ECHO_REPORT.txt'

In [249]:
document = open(filename, 'r').read()

In [250]:
document

'102920\t||||\t176\t||||\t2167\t||||\tECHO_REPORT\t||||\t2013-09-23 00:00:00.0\t||||\t\t||||\t\nPATIENT/TEST INFORMATION:\nIndication: Pericardial effusion. \nHeight: (in) 68\nWeight (lb): 184\nBSA (m2): 1.97 m2\nBP (mm Hg): 140/80\nHR (bpm): 70\nStatus: Inpatient\nDate/Time: [**2013-09-23**] at 10:20\nTest: TTE (Focused views)\nDoppler: No doppler\nContrast: None\nTechnical Quality: Adequate\n\n||||\t\t||||\t\nINTERPRETATION: \n\nFindings: \n\nLEFT ATRIUM: The left atrium is moderately dilated. The left atrium is elongated. \n\nRIGHT ATRIUM/INTERATRIAL SEPTUM: The right atrium is moderately dilated. \n\nLEFT VENTRICLE: There is severe symmetric left ventricular hypertrophy. The left ventricular cavity size is normal. Overall left ventricular systolic function is low normal (LVEF 50-55%). \n\nLV WALL MOTION: The following resting regional left ventricular wall motion abnormalities are seen: basal inferoseptal - hypokinetic; mid inferoseptal - hypokinetic; basal inferior - hypokinetic; 

In [258]:
abbr_index_list = list(task2_test_dict[filename.split('/')[-1]].keys())

In [259]:
abbr_index_list

[(31, 35),
 (176, 179),
 (194, 196),
 (213, 215),
 (287, 290),
 (746, 750),
 (762, 764),
 (1861, 1865),
 (2218, 2220)]

In [276]:
output = ''
last = 0
for var in abbr_index_list:
    begin = int(var[0])
    end = int(var[1])
    output = output + document[last:begin] + "@@@"
    last = end
output = output + document[last:len(document)]    

In [274]:
output

'102920\t||||\t176\t||||\t2167\t||||\t@@@_REPORT\t||||\t2013-09-23 00:00:00.0\t||||\t\t||||\t\nPATIENT/TEST INFORMATION:\nIndication: Pericardial effusion. \nHeight: (in) 68\nWeight (lb): 184\n@@@ (m2): 1.97 m2\n@@@ (mm Hg): 140/80\n@@@ (bpm): 70\nStatus: Inpatient\nDate/Time: [**2013-09-23**] at 10:20\nTest: @@@ (Focused views)\nDoppler: No doppler\nContrast: None\nTechnical Quality: Adequate\n\n||||\t\t||||\t\nINTERPRETATION: \n\nFindings: \n\nLEFT ATRIUM: The left atrium is moderately dilated. The left atrium is elongated. \n\nRIGHT ATRIUM/INTERATRIAL SEPTUM: The right atrium is moderately dilated. \n\nLEFT VENTRICLE: There is severe symmetric left ventricular hypertrophy. The left ventricular cavity size is normal. Overall left ventricular systolic function is low normal (@@@ 50-55%). \n\n@@@ WALL MOTION: The following resting regional left ventricular wall motion abnormalities are seen: basal inferoseptal - hypokinetic; mid inferoseptal - hypokinetic; basal inferior - hypokinetic;

In [275]:
document

'102920\t||||\t176\t||||\t2167\t||||\tECHO_REPORT\t||||\t2013-09-23 00:00:00.0\t||||\t\t||||\t\nPATIENT/TEST INFORMATION:\nIndication: Pericardial effusion. \nHeight: (in) 68\nWeight (lb): 184\nBSA (m2): 1.97 m2\nBP (mm Hg): 140/80\nHR (bpm): 70\nStatus: Inpatient\nDate/Time: [**2013-09-23**] at 10:20\nTest: TTE (Focused views)\nDoppler: No doppler\nContrast: None\nTechnical Quality: Adequate\n\n||||\t\t||||\t\nINTERPRETATION: \n\nFindings: \n\nLEFT ATRIUM: The left atrium is moderately dilated. The left atrium is elongated. \n\nRIGHT ATRIUM/INTERATRIAL SEPTUM: The right atrium is moderately dilated. \n\nLEFT VENTRICLE: There is severe symmetric left ventricular hypertrophy. The left ventricular cavity size is normal. Overall left ventricular systolic function is low normal (LVEF 50-55%). \n\nLV WALL MOTION: The following resting regional left ventricular wall motion abnormalities are seen: basal inferoseptal - hypokinetic; mid inferoseptal - hypokinetic; basal inferior - hypokinetic; 